In [17]:
%pip install aiohttp

Note: you may need to restart the kernel to use updated packages.


In [18]:
import os
import aiohttp
import asyncio
import pandas as pd
from dotenv import load_dotenv
from tqdm.notebook import tqdm
import time

In [19]:
time.strftime("%Y%m%d_%H%M%S")

'20251019_112625'

In [20]:
async def get_total_count(api_key):
    """전체 데이터 수를 조회하는 함수"""    
    url = f"http://openapi.seoul.go.kr:8088/{api_key}/json/tbLnOpendataRentV/1/1/2025/11680/강남구"    
    try:
        async with aiohttp.ClientSession() as session:
            async with session.get(url) as response:
                data = await response.json()        
                return data['tbLnOpendataRentV']['list_total_count']
    except Exception as e:
        print(f"데이터 수 조회 중 오류 발생: {e}")
        return 0

async def get_rent_data(start_idx, end_idx, api_key):
    """임대차 정보를 조회하는 함수"""    
    url = f"http://openapi.seoul.go.kr:8088/{api_key}/json/tbLnOpendataRentV/{start_idx}/{end_idx}/2025/11680/강남구"    
    try:
        async with aiohttp.ClientSession() as session:
            async with session.get(url) as response:
                if response.status == 200:
                    data = await response.json()
                    return data['tbLnOpendataRentV']['row']
                else:
                    print(f"API 오류 발생: 상태 코드 {response.status}")
                    return []
    except Exception as e:
        print(f"데이터 조회 중 오류 발생: {e}")
        return []

In [21]:
async def main():
    # .env 파일에서 환경변수 로드
    load_dotenv()
    api_key = os.getenv('SEOUL_LANDMARK_API')    

    # 전체 데이터 수 조회
    total_count = await get_total_count(api_key)
    print(f"전체 데이터 수: {total_count}")

    if total_count == 0:    
        raise Exception("데이터를 가져올 수 없습니다.")

    # 데이터를 저장할 리스트
    all_data = []

    # 페이지 크기를 1000으로 설정
    page_size = 1000

    # 전체 반복 횟수 계산 (1000건 단위로)
    total_iterations = (total_count + page_size - 1) // page_size

    # 1000페이지씩 데이터 조회 (tqdm으로 진행률 표시)
    progress_bar = tqdm(range(1, total_count + 1, page_size), 
                    total=total_iterations,
                    desc="데이터 수집 진행률",
                    unit="천건")

    for start in progress_bar:
        end = min(start + page_size - 1, total_count)
        data = await get_rent_data(start, end, api_key)
        all_data.extend(data)
        
        # 진행상황 업데이트 메시지 (천 단위로 표시)
        progress_bar.set_postfix(현재=f"{start//1000}k~{(end//1000)}k")

    # 데이터프레임 생성
    df = pd.DataFrame(all_data)

    # CSV 파일로 저장
    current_time = time.strftime("%Y%m%d_%H%M%S")
    filename = f"seoul_rent_data_{current_time}.csv"
    df.to_csv(filename, index=False, encoding='utf-8-sig')
    print(f"데이터가 {filename}에 저장되었습니다.")

    # 데이터 요약 정보 출력
    print("\n데이터 요약:")
    print(f"총 행 수: {len(df)}")
    print("\n컬럼 정보:")
    for col in df.columns:
        print(f"- {col}")

# 비동기 함수 실행
await main()

전체 데이터 수: 28837


데이터 수집 진행률:   0%|          | 0/29 [00:00<?, ?천건/s]

데이터가 seoul_rent_data_20251019_112710.csv에 저장되었습니다.

데이터 요약:
총 행 수: 28837

컬럼 정보:
- RCPT_YR
- CGG_CD
- CGG_NM
- STDG_CD
- STDG_NM
- LOTNO_SE
- LOTNO_SE_NM
- MNO
- SNO
- FLR
- CTRT_DAY
- RENT_SE
- RENT_AREA
- GRFE
- RTFE
- BLDG_NM
- ARCH_YR
- BLDG_USG
- CTRT_PRD
- NEW_UPDT_YN
- CTRT_UPDT_USE_YN
- BFR_GRFE
- BFR_RTFE


In [ ]:
import pandas as pd
df = pd.read_csv('seoul_rent_data_20251019_112710.csv')
df.head()
df = df.drop(columns=['RCPT_YR','CGG_CD'	,'CGG_NM',	'STDG_CD','LOTNO_SE'])
df.head()

In [25]:
# 컬럼명 매핑 딕셔너리 생성
column_mapping = {
    'STDG_NM': '법정동명',
    'LOTNO_SE_NM': '지번구분명',
    'MNO': '본번',
    'SNO': '부번',
    'FLR': '층',
    'CTRT_DAY': '계약일',
    'RENT_SE': '전월세구분',
    'RENT_AREA': '임대면적(㎡)',
    'GRFE': '보증금(만원)',
    'RTFE': '임대료(만원)',
    'BLDG_NM': '건물명',
    'ARCH_YR': '건축년도',
    'BLDG_USG': '건물용도',
    'CTRT_PRD': '계약기간',
    'NEW_UPDT_YN': '신규갱신여부',
    'CTRT_UPDT_USE_YN': '계약갱신권사용여부',
    'BFR_GRFE': '종전보증금',
    'BFR_RTFE': '종전임대료'
}

# 컬럼명 변경
df = df.rename(columns=column_mapping)

print("변경된 컬럼 목록:")
for col in df.columns:
    print(f"- {col}")

print("\n데이터 미리보기:")
df.head()

변경된 컬럼 목록:
- 법정동명
- 지번구분명
- 본번
- 부번
- 층
- 계약일
- 전월세구분
- 임대면적(㎡)
- 보증금(만원)
- 임대료(만원)
- 건물명
- 건축년도
- 건물용도
- 계약기간
- 신규갱신여부
- 계약갱신권사용여부
- 종전보증금
- 종전임대료

데이터 미리보기:


,법정동명,지번구분명,본번,부번,층,계약일,전월세구분,임대면적(㎡),보증금(만원),임대료(만원),건물명,건축년도,건물용도,계약기간,신규갱신여부,계약갱신권사용여부,종전보증금,종전임대료
0,압구정동,대지,397.0,0.0,11.0,20251017,전세,74.40,85000,0,미성2차,1987.0,아파트,25.11~27.11,갱신,NaN,70000.0,NaN
1,역삼동,대지,774.0,33.0,3.0,20251017,전세,40.42,35000,0,뮤지컬파크,2015.0,연립다세대,25.11~27.11,신규,NaN,NaN,NaN
2,역삼동,대지,825.0,24.0,11.0,20251017,월세,22.85,18000,30,강남역 효성해링턴 타워 더퍼스트,2014.0,오피스텔,25.10~26.10,신규,NaN,NaN,NaN
3,청담동,대지,125.0,0.0,2.0,20251017,전세,84.78,70000,0,범신칼릭스빌,2005.0,아파트,25.12~27.12,신규,NaN,NaN,NaN
4,세곡동,대지,525.0,0.0,9.0,20251017,월세,59.97,9683,17,강남데시앙파크,2011.0,아파트,25.11~27.10,NaN,NaN,NaN,NaN


In [28]:
# 주소 컬럼 생성
def create_address(row):
    # 기본 주소 구성
    address = f"서울특별시 강남구 {row['법정동명']}"
    
    # 지번구분명이 '산'인 경우 추가
    if row['지번구분명'] == '산':
        address += f" {row['지번구분명']}"
    
    # 본번 추가    
    try:    
        address += f" {int(row['본번'])}"  # 정수로 변환하여 불필요한 소수점 제거
    except:
        address += ''
    
    # 부번이 0이 아닌 경우에만 추가
    try:
        if row['부번'] != 0:
            address += f"-{int(row['부번'])}"
    except:
        address += ''
        
    return address

# 주소 컬럼 추가
df['주소'] = df.apply(create_address, axis=1)

# 새로운 컬럼 순서 지정 (주소를 앞쪽으로 이동)
columns = ['주소'] + [col for col in df.columns if col != '주소']
df = df[columns]

df.head()

,주소,법정동명,지번구분명,본번,부번,층,계약일,전월세구분,임대면적(㎡),보증금(만원),임대료(만원),건물명,건축년도,건물용도,계약기간,신규갱신여부,계약갱신권사용여부,종전보증금,종전임대료
0,서울특별시 강남구 압구정동 397,압구정동,대지,397.0,0.0,11.0,20251017,전세,74.40,85000,0,미성2차,1987.0,아파트,25.11~27.11,갱신,NaN,70000.0,NaN
1,서울특별시 강남구 역삼동 774-33,역삼동,대지,774.0,33.0,3.0,20251017,전세,40.42,35000,0,뮤지컬파크,2015.0,연립다세대,25.11~27.11,신규,NaN,NaN,NaN
2,서울특별시 강남구 역삼동 825-24,역삼동,대지,825.0,24.0,11.0,20251017,월세,22.85,18000,30,강남역 효성해링턴 타워 더퍼스트,2014.0,오피스텔,25.10~26.10,신규,NaN,NaN,NaN
3,서울특별시 강남구 청담동 125,청담동,대지,125.0,0.0,2.0,20251017,전세,84.78,70000,0,범신칼릭스빌,2005.0,아파트,25.12~27.12,신규,NaN,NaN,NaN
4,서울특별시 강남구 세곡동 525,세곡동,대지,525.0,0.0,9.0,20251017,월세,59.97,9683,17,강남데시앙파크,2011.0,아파트,25.11~27.10,NaN,NaN,NaN,NaN


In [ ]:
# 서울시에 있는 법정동별로 api 호출해서 조회하는 기능
# 법정동 코드와 명칭 리스트가 필요
# 사용자가 해당 법정동 명을 입력 또는 선택하면
# api 호출 주소를 만들어서 조회
# 결과는 데이터프레임 형태의 변수에 저장( 기존 컬럼정보를 이용해서 주소컬럼도 생성되어야 한다)